# Data preprocessing

## Load text

In [ ]:
# load text
filename = 'metamorphosis_clean.txt'
file = open(filename, 'rt')
text = file.read()
file.close()



In [1]:
# Assume our input is " input1   target1
#                       input2   target2"

## Generate pairs and normalize, filter the pairs

In [ ]:
# normalize
def normalize(s):
    s = s.lower().strip()
    s = re.sub(r"([,.!?])", r" \1 ", s)
    s = re.sub(r"[^a-zA-Z,.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    #s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [ ]:
# generate
lines = text.strip().split('\n')
pairs = [[normalize(s) for s in l.split('\t')] for l in lines]
# pairs = list(zip(input_sen,target_sen))
# pairs = [list(i) for i in pairs]

In [ ]:
# filter pairs by length of the 
def filter_pairs(pairs, MIN_LENGTH, MAX_LENGTH):
    filtered_pairs = []
    print ("Initially read %d pairs" % len(pairs))
    for pair in pairs:
        if len(pair[0]) >= MIN_LENGTH and len(pair[0]) <= MAX_LENGTH \
            and len(pair[1]) >= MIN_LENGTH and len(pair[1]) <= MAX_LENGTH:
                filtered_pairs.append(pair)
    print ("filtered pairs %d" %len(filtered_pairs))
    return filtered_pairs

pairs = filter_pairs(pairs, 1, 15)

# Vocabulary Class

In [ ]:
PAD_token = 0
SOS_token = 1
EOS_token = 2

class Lang:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "PAD", 1: "SOS", 2: "EOS"}
        self.n_words = 3 # Count default tokens

    def index_words(self, sentence):
        for word in sentence.split(' '):
            self.index_word(word)

    def index_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed: return
        self.trimmed = True
        
        keep_words = []
        
        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words %s / %s = %.4f' % (
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "PAD", 1: "SOS", 2: "EOS"}
        self.n_words = 3 # Count default tokens

        for word in keep_words:
            self.index_word(word)

In [ ]:
PAD = 0
SOS = 1
EOS = 2

class Lang:
    def __intial__(self,name,trim_Bool):
        self.name = name
        self.trim = trim_bool
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0:'PAD',1:'SOS',2:'EOS'}
        self.n_words = 3
        
    def index_words(sentence):
        return [index_word(s) for s in sentence.split(' ')]
    
    def index_word(word):
        if word not in word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words+=1
        else:
            self.word2count[word] +=1
    
    def trim(self, min_count):
        if not self.trim_bool: return
        keep_word = []
        for k,v in self.word2count.items():
            if v>=min_count:
                keep_word.append(k)
                
        
            

In [ ]:
# build vocabulary
vocab = Lang()
for pair in pairs:
    vocab.index_words(pair[0])
    vocab.index_words(pair[1])

In [ ]:
# filter out the non-frequent word
MIN_COUNT = 3

vocab.trim(MIN_COUNT)


In [ ]:
# Remove those pairs with unknown word
keep_pairs = []

for pair in pairs:
    input_sentence = pair[0]
    output_sentence = pair[1]
    keep_input = True
    keep_output = True
    
    for word in input_sentence.split(' '):
        if word not in input_lang.word2index:
            keep_input = False
            break

    for word in output_sentence.split(' '):
        if word not in output_lang.word2index:
            keep_output = False
            break

    # Remove if pair doesn't match input and output conditions
    if keep_input and keep_output:
        keep_pairs.append(pair)

print("Trimmed from %d pairs to %d, %.4f of total" % (len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
pairs = keep_pairs

# Generate Batch

In [ ]:
def indexes_from_sentence(lang, sentence):
    return [vocab.word2index[word] for word in sentence.split(' ')] + [EOS_token]

In [ ]:
def pad_seq(seq, max_length):
    seq += [PAD_token for i in range(max_length - len(seq))]
    return seq

In [ ]:
# generate batches
def random_batch(batch_size):
    input_seqs = []
    target_seqs = []

    # Choose random pairs
    for i in range(batch_size):
        pair = random.choice(pairs, replace = False)
        input_seqs.append(indexes_from_sentence(vocab, pair[0]))
        target_seqs.append(indexes_from_sentence(vocab, pair[1]))
    
    # Zip into pairs, sort by length (descending), unzip
    # Note that normally, we sort by input seqs, because there is a bi-directional LSTM/GRU there. For target, single 
    # direction does not always need pack_padeed
    seq_pairs = sorted(zip(input_seqs, target_seqs), key=lambda p: len(p[0]), reverse=True)
    input_seqs, target_seqs = zip(*seq_pairs)
    
    # For input and target sequences, get array of lengths and pad with 0s to max length
    input_lengths = [len(s) for s in input_seqs]
    input_padded = [pad_seq(s, max(input_lengths)) for s in input_seqs]
    target_lengths = [len(s) for s in target_seqs]
    target_padded = [pad_seq(s, max(target_lengths)) for s in target_seqs]

    # Turn padded arrays into (batch_size x max_len) tensors, transpose into (max_len x batch_size)
    input_var = Variable(torch.LongTensor(input_padded)).transpose(0, 1)
    target_var = Variable(torch.LongTensor(target_padded)).transpose(0, 1)
    
    if USE_CUDA:
        input_var = input_var.cuda()
        target_var = target_var.cuda()
        
    return input_var, input_lengths, target_var, target_lengths

# Encoder

## Encoder for classification

In [ ]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(x))
        
        #embedded = [sent len, batch size, emb dim]
        
        output, (hidden, cell) = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim * num directions]
        #hidden = [num layers * num directions, batch size, hid. dim]
        #cell = [num layers * num directions, batch size, hid. dim]
        
        # Here only concatenate the two hidden states (bi-directions) of the layer 2. 
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        
        # note hidden[-2,:,:] has a size of [batch size, hid. dim]        
        #hidden [batch size, hid. dim * num directions]
            
        return self.fc(hidden.squeeze(0))

## Encoder for seq2seq

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1, dropout=0.1):
        super(EncoderRNN, self).__init__()
        
        self.input_size = input_size  # input_size = vocaburary_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.embedding_size = hidden_size
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.gru = nn.GRU(embedding_size, hidden_size, n_layers, dropout=self.dropout, bidirectional=True)
        
    def forward(self, input_seqs, input_lengths, hidden=None):
        '''' Note: we run this all at once (over multiple batches of multiple sequences)''''
        # input_seqs : max_len,batch_size ; embedded: max_len,batch_size,embedding(hidden_size)
        embedded = self.embedding(input_seqs)
        # packed: max_len,batch_size,embedding(hidden_size)
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # hidden: num_layer*num_direction,batch_size,hidden_size; 
        # outputs: max_len,batch_size,num_direct*hidden_size
        # hidden: default to 0 if not provided 
        outputs, hidden = self.gru(packed, hidden)
        outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs) # unpack (back to padded)
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:] # Sum bidirectional outputs
        #outputs: max_len,batch_size,hidden_size
        #hidden: 2,batch_size,hidden_size
        return outputs, hidden

## Encoder for CNN text - classification

In [ ]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv_0 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[0],embedding_dim))
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[1],embedding_dim))
        self.conv_2 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[2],embedding_dim))
        self.fc = nn.Linear(len(filter_sizes)*n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [sent len, batch size]
        
        x = x.permute(1, 0)
                
        #x = [batch size, sent len]
        
        embedded = self.embedding(x)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
            
        #conv_n = [batch size, n_channels, sent len - filter_sizes[n]]
        
        pooled_0 = F.max_pool1d(conv_0, conv_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conv_1, conv_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conv_2, conv_2.shape[2]).squeeze(2)
        
        #pooled_n = [batch size, n_channels]
        
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))

        #cat = [batch size, n_channels * len(filter_sizes)]
            
        return self.fc(cat)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        self.conv1 = nn.Conv2d(1,32,5)
        self.conv1_bn = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(32,64,5)
        self.pool2 = nn.MaxPool2d(2,2)
        self.conv3 = nn.Conv2d(64,128,5)
        self.dropout1= nn.Dropout2d(p = 0.6)
        self.pool3 = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(,300)
        self.dropout2 = nn.Dropout(0.7)
        self.fc2 = nn.Linear(300,136)
        
    def forward(self,x):
        x = self.pool1(F.relu(self.conv1_bn(self.conv1(x))))
        x = self.pool2(F.relu(self.conv2_bn(self.conv2(x))))
        x = x.view(x.size(0),-1)
        

## Encoder for CNN - image

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        
        # out = (input + 2*padding - kernel_size)/stride + 1

        # 1 input image channel (grayscale), 32 output channels/feature maps, 5x5 square convolution kernel
        # input 224, conv1_output (32,220,220), pool1_out (32,110,110)
        
        
        
        
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv1_bn = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2,2)
        # input 110, conv2_out (64,107,107), pool2_out (64,53,53)
        self.conv2 = nn.Conv2d(32, 64, 4)
        self.conv2_bn = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2,2)
        # input 53, conv3_out (128,51,51), pool3_out (128,25,25)
        self.conv3 = nn.Conv2d(64, 128,3)
        self.dropout1 = nn.Dropout2d(p =0.6)
        self.pool3 = nn.MaxPool2d(2,2)
        
        # flattened array 128*25*25 = 80000
        #self.dropout1 = nn.Dropout(p =0.2)
        self.fc1 = nn.Linear(128*25*25,300)
        self.fc1_bn = nn.BatchNorm1d(300)
        self.dropout2 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(300,136)
        
        
    def forward(self, x):
        
        x = self.pool1(F.relu(self.conv1_bn(self.conv1(x))))
        x = self.pool2(F.relu(self.conv2_bn(self.conv2(x))))
        x = self.pool3(F.relu(self.dropout1(self.conv3(x))))
        x = x.view(x.size(0),-1)
        #x = self.dropout1(x)
        x = F.relu(self.fc1_bn(self.fc1(x)))
        x = self.dropout2(x)
        x = self.fc2(x)

## Encoder for Amex solution

In [ ]:
class solution_encoder(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, output_size, dropout, n_filters, filter_size):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.gru = nn.GRU(embedding_size, hidden_size, n_layers =1, bidirectional = True)
        self.conv0 = nn.Conv2d(1, n_filters, (filter_size[0],hidden_size))
        self.conv1 = nn.Conv2d(1, n_filters, (filter_size[1],hidden_size))
        self.fc1 = nn.Linear(len(filter_size)*n_filters, output_size)
        self.fc2 = nn.Linear((output_size+hidden_size),class_size)
        
    def forward(input_seq, input_lengths, hidden=None):
        embedded  = self.embedding(input) # S*B*E
        pack = utils.rnn.pack_padded_sequence(embedded, input_lengths)
        outputs, hidden = self.gru(pack, hidden)
        outputs, outputs_length = utils.rnn.pad_packed_sequence(outputs) #outputs  # S*B*2H, Hidden # 2*B*H
        outputs = outputs[:,:,:self.hidden_size] + [:,:,self.hidden_size:]   # outputs  S*B*H
        hidden = hidden[-2,:,:] + hidden[-1,:,:] # B*H
        outputs = outputs.permute(1,0,2)  B*S*H
        outputs = outputs.unsqueeze(1)   B*1*S*H
        return outputs, hidden
        
class solution_decoder(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, output_size, dropout, n_filters, filter_size):
        super().__init__()
        
        self.conv0 = nn.Conv2d(1, n_filters, (filter_size[0],hidden_size))
        self.conv1 = nn.Conv2d(1, n_filters, (filter_size[1],hidden_size))
        self.fc1 = nn.Linear(len(filter_size)*n_filters, output_size)
    
    def forward(encoder_output, hidden)
        conv0 = F.relu(conv_0(outputs).squeeze(3))   #  B*n_filters*(S-filter_size)*1
        conv1 = F.relu(conv_1(outputs).squeeze(3))   #  B*n_filters*(S-filter_size)*1
        pool0 = F.max_pool1d(conv0, conv0.shape[2]).squeeze(2)
        pool1 = F.max_pool1d(conv1, conv1.shape[2]).squeeze(2)  # B*n_filters
        out = torch.cat((pool0,pool1),dim = 1)   # B*2n_filters
        out = self.fc1(out)   # B*output_size
        hidden = hidden # B*hidden_size
        final_in = torch.cat((out,hidden), dim = 1) # B* (hidden_size+output_size)
        final_out = self.fc2(final_in)
        final_out = F.log_softmax(final_out)
        
        return final_out
        

# Decoder

## Simple decoder

In [ ]:
# for simple decoder we can run  all timesteps at once
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        # output_size = vocaburary_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        
    def forward(self, input_seq, last_hidden):
        '''' Note: we run this all steps at once (over multiple batches of multiple sequences)''''
        # embedded: max_len,batch_size,embedding_size(hidden_size)
        embedded = self.embedding(input_seq)
        # outputs: max_len,batch_size, 1*hidden_size; hidden: 1*1, batch_size, hidden_size
        outputs, hidden = self.gru(embedded,last_hidden)
        # outputs: batch_size, max_len, 1*hidden_size
        outputs = outputs.permute(1,0,2)
        # outputs: batch_size, max_len, output_size(vocaburary_size)
        outputs = self.out(outputs)
        ouputs = F.log_softmax(outputs, dim =2)
    
    def sample(self,inputs,states = None, max_length = 20):
        '''' Note: we run this one step at a time ''''
        created_wordid = []
        A = random.randint(10,max_length)
        for i in range(A):
            # inputs: seq_len(1), batch_len(1), embedding_size(hidden_size); states: 1,1,hidden_size
            output, states = self.GRU(inputs,states)
            #output: batch(1),seq(1),hidden_size
            output = output.permute(1,0,2)
            #output: batch(1),output_size(vocab_size)
            output = self.out(output.squeeze(1))
            _,wordid = output.max(dim = 1)
            prediction = wordid.item()
            created_wordid.append(prediction)
            inputs = self.embed(wordid)
            inputs = inputs.view(1,1,-1)
        return created_wordid
            
            

## Attention decoder (Luang's attention)

- Bahaum:   score is determined by last hidden state (t-1) and context
- Luang:   score is determined by current RNN state (t) and all encoder outputs

In [ ]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=dropout)
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        
        # Choose attention model
        if attn_model != 'none':
            self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_seq, last_hidden, encoder_outputs):
        '''' Note: we run this one step at a time ''''

        # Get the embedding of the current input word (last output word)
        # input_seq: batch_size, seq_len(1)
        batch_size = input_seq.size(0)
        # embedded: batch_size, seq_len(1),hidden_size
        embedded = self.embedding(input_seq)
        embedded = self.embedding_dropout(embedded)
        # embedded: seq_len(1), batch_size, hidden_size
        embedded = embedded.view(1, batch_size, self.hidden_size) 

        # Get current hidden state from input word and last hidden state
        # last_hidden/hidden: num_layer*direction(1),batch_size, hidden_size
        # rnn_output: seq_len(1), batch_size, hidden_size
        rnn_output, hidden = self.gru(embedded, last_hidden)

        # Calculate attention from current RNN state and all encoder outputs;
        # apply to encoder outputs to get weighted average
        # rnn_output: seq_len(1), batch_size, hidden_size
        # encoder_outputs: seq_len_encoder, batch_size, hidden_size
        # attb_weights = batch_size, 1, seq_len_encoder
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # context: Batch_size x 1 x hidden_size
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1)) 

        # Attentional vector using the RNN hidden state and context vector
        # concatenated together (Luong eq. 5)
        rnn_output = rnn_output.squeeze(0) # S=1 x B x N -> batch_size, hidden_size
        context = context.squeeze(1)       # B x S=1 x N -> batch_size, hidden_size
        concat_input = torch.cat((rnn_output, context), 1) # concat_input: batch_size, 2*hidden_size
        # feed input to a linear layer called concat
        concat_output = F.tanh(self.concat(concat_input))   # concat_output: batch_size, hidden_size

        # Finally predict next token (Luong eq. 6, without softmax)
        # output: batch_size, output_size(vocaburary_size)
        output = self.out(concat_output)

        # Return final output, hidden state, and attention weights (for visualization)
        return output, hidden, attn_weights

In [ ]:
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        
        self.method = method
        self.hidden_size = hidden_size
        
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)

    def forward(self, hidden, encoder_outputs):
        # hidden = rnn_output: seq_len(1), batch_size, hidden_size
        # encoder_outputs: seq_len_encoder,batch_size,hidden_size
        max_len = encoder_outputs.size(0)
        this_batch_size = encoder_outputs.size(1)

        # Create variable to store attention energies
        # attn_energies: batch_size, seq_len_encoder
        attn_energies = Variable(torch.zeros(this_batch_size, max_len))

        if USE_CUDA:
            attn_energies = attn_energies.cuda()

        # For each batch of encoder outputs
        for b in range(this_batch_size):
            # Calculate energy for each encoder output
            for i in range(max_len):
                # hidden[:, b]: 1,hidden_size
                # encoder_outputs[i, b].unsqueeze(0): 1, hidden_size
                attn_energies[b, i] = self.score(hidden[:, b], encoder_outputs[i, b].unsqueeze(0))

        # Normalize energies to weights in range 0 to 1, resize to  Batch_size, 1, seq_len_encoder
        return F.softmax(attn_energies).unsqueeze(1) 
    
    def score(self, hidden, encoder_output):
        
        if self.method == 'dot':
            energy = hidden.dot(encoder_output)
            return energy
        
        elif self.method == 'general':
            energy = self.attn(encoder_output)
            energy = hidden.dot(energy)
            return energy
        
        elif self.method == 'concat':
            energy = self.attn(torch.cat((hidden, encoder_output), 1))
            energy = self.v.dot(energy)
            return energy

# Train the net work

In [ ]:
def train(input_batches, input_lengths, target_batches, target_lengths, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    
    # Zero gradients of both optimizers
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    loss = 0 # Added onto for each word

    # Run words through encoder
    encoder_outputs, encoder_hidden = encoder(input_batches, input_lengths, None)
    
    # Prepare input and output variables
    decoder_input = Variable(torch.LongTensor([SOS_token] * batch_size))
    decoder_hidden = encoder_hidden[:decoder.n_layers] # Use last (forward) hidden state from encoder

    max_target_length = max(target_lengths)
    # all_decoder_outputs: max_target_seq, batch_size, decoder
    all_decoder_outputs = Variable(torch.zeros(max_target_length, batch_size, decoder.output_size))

    # Move new Variables to CUDA
    if USE_CUDA:
        decoder_input = decoder_input.cuda()
        all_decoder_outputs = all_decoder_outputs.cuda()

    # Run through decoder one time step at a time
    for t in range(max_target_length):
        decoder_output, decoder_hidden, decoder_attn = decoder(
            decoder_input, decoder_hidden, encoder_outputs
        )

        all_decoder_outputs[t] = decoder_output
        decoder_input = target_batches[t] # Next input is current target

    # Loss calculation and backpropagation
    """""
    loss = masked_cross_entropy(
        all_decoder_outputs.transpose(0, 1).contiguous(), # -> batch x seq
        target_batches.transpose(0, 1).contiguous(), # -> batch x seq
        target_lengths
    )
    """""
    loss = cross_entropy(all_decoder_outputs.transpose(0, 1).contiguous(),target_batches.transpose(0, 1).contiguous()
                        , ignore_index = 0)
    loss.backward()
    
    # Clip gradient norms
    ec = torch.nn.utils.clip_grad_norm(encoder.parameters(), clip)
    dc = torch.nn.utils.clip_grad_norm(decoder.parameters(), clip)

    # Update parameters with optimizers
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.data[0], ec, dc

In [ ]:
attn_model = 'dot'
hidden_size = 500
n_layers = 2
dropout = 0.1
batch_size = 100
batch_size = 50

# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 0.5
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_epochs = 50000
epoch = 0
plot_every = 20
print_every = 100
evaluate_every = 1000

# Initialize models
encoder = EncoderRNN(input_lang.n_words, hidden_size, n_layers, dropout=dropout)
decoder = LuongAttnDecoderRNN(attn_model, hidden_size, output_lang.n_words, n_layers, dropout=dropout)

# Initialize optimizers and criterion
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
criterion = nn.CrossEntropyLoss()

In [ ]:
def evaluate(input_seq, max_length=MAX_LENGTH):
    input_lengths = [len(input_seq)]
    input_seqs = [indexes_from_sentence(input_lang, input_seq)]
    input_batches = Variable(torch.LongTensor(input_seqs), volatile=True).transpose(0, 1)
    
    if USE_CUDA:
        input_batches = input_batches.cuda()
        
    # Set to not-training mode to disable dropout
    encoder.train(False)
    decoder.train(False)
    
    # Run through encoder
    encoder_outputs, encoder_hidden = encoder(input_batches, input_lengths, None)

    # Create starting vectors for decoder
    decoder_input = Variable(torch.LongTensor([SOS_token]), volatile=True) # SOS
    decoder_hidden = encoder_hidden[:decoder.n_layers] # Use last (forward) hidden state from encoder
    
    if USE_CUDA:
        decoder_input = decoder_input.cuda()

    # Store output words and attention states
    decoded_words = []
    decoder_attentions = torch.zeros(max_length + 1, max_length + 1)
    
    # Run through decoder
    for di in range(max_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs
        )
        decoder_attentions[di,:decoder_attention.size(2)] += decoder_attention.squeeze(0).squeeze(0).cpu().data

        # Choose top word from output
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]
        if ni == EOS_token:
            decoded_words.append('<EOS>')
            break
        else:
            decoded_words.append(output_lang.index2word[ni])
            
        # Next input is chosen word
        decoder_input = Variable(torch.LongTensor([ni]))
        if USE_CUDA: decoder_input = decoder_input.cuda()

    # Set back to training mode
    encoder.train(True)
    decoder.train(True)
    
    return decoded_words, decoder_attentions[:di+1, :len(encoder_outputs)]